In [1]:
from ECG_helpers_tf import *


Using TensorFlow backend.


In [6]:
def generate_model(model_type, n_layers, n_nodes, target, resize_factor, X):
    model = Sequential()
    if model_type == 'Conv1D':
        conv_size=3
        pool_size=2
        pool=False
        model.add(Conv1D(n_nodes, conv_size, input_shape=X.shape[1:], activation='relu', padding='same'))
        if pool:
            model.add(MaxPooling1D(pool_size))
        for l in range(n_layers-2):
            model.add(Conv1D(n_nodes, conv_size, activation='relu', padding='same'))
            if pool:
                model.add(MaxPooling1D(pool_size))
        model.add(Conv1D(n_nodes, conv_size, activation='relu', padding='same'))
        model.add(GlobalMaxPooling1D())
    else:
        layer=globals()[model_type]
        model.add(layer(n_nodes, input_shape=X.shape[1:], return_sequences = True))
        for l in range(n_layers-2):
            model.add(layer(n_nodes, return_sequences=True))
        model.add(layer(n_nodes))
    model.add(Dense(1, activation=dict_activation[target]))
    return model


In [41]:
model_type = 'SimpleRNN' #available: 'SimpleRNN', 'GRU', 'LSTM'
n_layers = 5 #must be >= 2
n_nodes = 128
target = 'Age' #available: 'Age', 'Sex'
optimizer_name = 'Adam'
learning_rate = 0.0000001
n_epochs=2
resize_factor=30

In [42]:
#load data
for fold in folds:
    globals()['demo_'+fold] = pd.read_csv(path_store + 'demo_' + fold + '.csv', header=0, index_col=0)
    globals()['X_'+fold] = np.load(path_store + 'X_' + fold + '.npy') #.astype(float) did not make a difference

#resize data   
for fold in folds:
    globals()['X_'+fold] = resize_X_by_timesteps(globals()['X_'+fold],resize_factor)


In [50]:
model = generate_model(model_type, n_layers, n_nodes, target, resize_factor, X_train)
opt = globals()[optimizer_name](lr=learning_rate)
model.compile(optimizer=opt, loss=dict_loss[target], metrics=dict_metrics[target])


In [51]:
history = model.fit(X_train, np.array(demo_train[target])/100, epochs=n_epochs, batch_size=128, validation_data=[X_val,np.array(demo_val[target])])


Train on 23463 samples, validate on 2933 samples
Epoch 1/2
 6656/23463 [=======>......................] - ETA: 14s - loss: nan - R_squared: nan - root_mean_squared_error: inf

KeyboardInterrupt: 

In [48]:
pred_val = model.predict(X_val).squeeze() #debug
print(pred_val[:200]) #debug


[ 6.00717723e-01  9.47190225e-01  5.52913189e-01 -5.45983195e-01
 -1.40489638e-02  1.03984785e+00  1.84948444e-01 -4.83200759e-01
  1.97146726e+00  4.87474918e-01  8.40528429e-01  5.77617660e-02
  3.93649489e-01  6.61190093e-01 -2.62115197e-03  8.83288145e-01
  2.08961034e+00 -9.27420855e-01  4.40356165e-01 -9.54556167e-01
  6.88228369e-01 -9.00484204e-01 -3.47562045e-01 -1.44477531e-01
 -9.06137347e-01 -1.83027017e+00 -2.32051626e-01  1.08436048e-01
 -1.40304655e-01 -6.58076823e-01  2.15107292e-01  1.23915505e+00
  4.71362203e-01  2.23422915e-01  1.72778055e-01  3.58815581e-01
  7.50274956e-01  1.36091590e+00 -5.09263277e-01  6.29131317e-01
 -1.44748539e-01  2.45887667e-01 -1.68260217e-01 -9.75245982e-03
  2.32111961e-02  4.68482524e-01  9.61789370e-01  8.91611725e-03
  3.66083756e-02 -2.93847248e-02 -3.37150127e-01  6.29379034e-01
  9.19044852e-01  7.59180665e-01  5.06118774e-01 -6.61204219e-01
 -9.45230961e-01  6.36745155e-01  1.54073787e+00 -8.66190612e-01
  9.20689821e-01  8.12123

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 200, 16)           592       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 16)           784       
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,393
Trainable params: 1,393
Non-trainable params: 0
_________________________________________________________________


In [19]:
X_train.shape

(23463, 200, 12)

In [20]:
X_train[0,:,1]

array([  8.33333333,  10.33333333,  10.66666667,   7.33333333,
         4.33333333,   6.33333333,   9.33333333,   8.33333333,
         6.        ,   5.33333333,   7.        ,  10.        ,
        10.66666667,   5.33333333,   7.33333333,   7.        ,
         6.        ,   7.33333333,   7.33333333,   7.33333333,
         7.33333333,   7.33333333,   6.        ,   5.        ,
         8.33333333,   7.66666667,   9.        ,   9.33333333,
         7.33333333,   5.66666667,   9.66666667,   7.33333333,
         5.        ,   5.33333333,   4.66666667,   4.        ,
         5.        ,   6.33333333,   6.        ,   5.66666667,
         6.        ,   5.33333333,   6.33333333,   9.        ,
        11.        ,  14.        ,  16.        ,  17.        ,
        17.66666667,  19.33333333,  21.        ,  22.66666667,
        18.66666667,  19.33333333,  21.        ,  19.33333333,
        18.66666667,  17.66666667,  13.66666667,  11.33333333,
         6.        ,   2.66666667,   2.        ,   0.  

In [21]:
y_train.head

NameError: name 'y_train' is not defined

In [22]:
demo_test.shape

(2933, 2)

In [23]:
demo_test.head

<bound method NDFrame.head of           Age  Sex
1188737  53.0  1.0
5044789  68.0  0.0
5926641  52.0  1.0
1982396  49.0  0.0
1588749  59.0  0.0
4000447  60.0  0.0
3617515  56.0  1.0
1469586  68.0  1.0
2272018  63.0  1.0
2831465  49.0  1.0
5505831  46.0  1.0
5759987  64.0  1.0
4154318  64.0  0.0
1618435  65.0  0.0
2160884  49.0  1.0
4287038  60.0  1.0
3956730  67.0  0.0
2287423  58.0  0.0
5240306  52.0  0.0
3330962  54.0  0.0
1468202  50.0  1.0
4414911  54.0  0.0
4189116  66.0  0.0
2652050  47.0  0.0
2027513  56.0  1.0
5636630  67.0  0.0
3791239  55.0  1.0
4924738  48.0  0.0
1351865  51.0  1.0
4558217  60.0  1.0
...       ...  ...
5937622  54.0  0.0
2259907  64.0  0.0
5028059  57.0  0.0
1003225  52.0  1.0
1273810  57.0  0.0
1153556  43.0  0.0
4395868  68.0  0.0
1523848  58.0  0.0
3085700  57.0  1.0
1667951  61.0  1.0
2131482  45.0  1.0
2473692  51.0  0.0
6024409  59.0  1.0
5243508  54.0  0.0
5509011  45.0  0.0
3532085  61.0  0.0
5457212  60.0  0.0
1854093  55.0  1.0
5911589  58.0  0.0
5